# Affinage des modèles Open AI

Ce carnet est basé sur les directives actuelles fournies dans la documentation [Affinage](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) d'Open AI.

L'affinage améliore les performances des modèles de base pour votre application en les réentraînant avec des données supplémentaires et un contexte pertinent pour ce cas d'utilisation ou scénario spécifique. Notez que les techniques d'ingénierie de prompt comme le _few shot learning_ et la _génération augmentée par récupération_ vous permettent d'améliorer le prompt par défaut avec des données pertinentes pour améliorer la qualité. Cependant, ces approches sont limitées par la taille maximale de la fenêtre de tokens du modèle de base ciblé.

Avec l'affinage, nous réentraînons effectivement le modèle lui-même avec les données requises (ce qui nous permet d'utiliser beaucoup plus d'exemples que ce qui peut tenir dans la fenêtre maximale de tokens) - et déployons une version _personnalisée_ du modèle qui n'a plus besoin d'avoir des exemples fournis au moment de l'inférence. Cela améliore non seulement l'efficacité de notre conception de prompt (nous avons plus de flexibilité pour utiliser la fenêtre de tokens pour d'autres choses) mais améliore potentiellement aussi nos coûts (en réduisant le nombre de tokens que nous devons envoyer au modèle au moment de l'inférence).

L'affinage comporte 4 étapes :
1. Préparer les données d'entraînement et les télécharger.
1. Lancer le travail d'entraînement pour obtenir un modèle affiné.
1. Évaluer le modèle affiné et itérer pour la qualité.
1. Déployer le modèle affiné pour l'inférence une fois satisfait.

Notez que tous les modèles de base ne supportent pas l'affinage - [consultez la documentation OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pour les informations les plus récentes. Vous pouvez également affiner un modèle déjà affiné. Dans ce tutoriel, nous utiliserons `gpt-35-turbo` comme modèle de base cible pour l'affinage.

---


### Étape 1.1 : Préparez votre jeu de données

Construisons un chatbot qui vous aide à comprendre le tableau périodique des éléments en répondant aux questions sur un élément avec un limerick. Dans _ce_ tutoriel simple, nous allons simplement créer un jeu de données pour entraîner le modèle avec quelques exemples de réponses montrant le format attendu des données. Dans un cas d'utilisation réel, vous devrez créer un jeu de données avec beaucoup plus d'exemples. Vous pourrez également utiliser un jeu de données ouvert (pour votre domaine d'application) s'il en existe un, et le reformater pour l'utiliser dans l'affinage.

Puisque nous nous concentrons sur `gpt-35-turbo` et recherchons une réponse en un seul tour (complétion de chat), nous pouvons créer des exemples en utilisant [ce format suggéré](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) reflétant les exigences de complétion de chat d'OpenAI. Si vous attendez un contenu conversationnel multi-tours, vous utiliseriez le [format d'exemple multi-tours](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) qui inclut un paramètre `weight` pour indiquer quels messages doivent être utilisés (ou non) dans le processus d'affinage.

Nous utiliserons ici le format plus simple en un seul tour pour notre tutoriel. Les données sont au format [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) avec 1 enregistrement par ligne, chacun représenté comme un objet JSON formaté. L'extrait ci-dessous montre 2 enregistrements en exemple - voir [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pour l'ensemble complet d'exemples (10 exemples) que nous utiliserons pour notre tutoriel d'affinage. **Remarque :** Chaque enregistrement _doit_ être défini sur une seule ligne (non réparti sur plusieurs lignes comme c'est typique dans un fichier JSON formaté)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Dans un cas d'utilisation réel, vous aurez besoin d'un ensemble d'exemples beaucoup plus grand pour de bons résultats - le compromis sera entre la qualité des réponses et le temps/coûts pour l'affinage. Nous utilisons un petit ensemble afin de pouvoir compléter l'affinage rapidement pour illustrer le processus. Voir [cet exemple du OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pour un tutoriel d'affinage plus complexe.


---

### Étape 1.2 Téléchargez votre jeu de données

Téléchargez les données en utilisant l'API Files [comme décrit ici](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Notez que pour exécuter ce code, vous devez d'abord avoir effectué les étapes suivantes :
 - Installer le package Python `openai` (assurez-vous d'utiliser une version >=0.28.0 pour les dernières fonctionnalités)
 - Définir la variable d'environnement `OPENAI_API_KEY` avec votre clé API OpenAI
Pour en savoir plus, consultez le [guide d'installation](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) fourni pour le cours.

Maintenant, exécutez le code pour créer un fichier à télécharger à partir de votre fichier JSONL local.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Étape 2.1 : Créer la tâche de fine-tuning avec le SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Étape 2.2 : Vérifier le statut du travail

Voici quelques actions que vous pouvez effectuer avec l’API `client.fine_tuning.jobs` :
- `client.fine_tuning.jobs.list(limit=<n>)` - Lister les n derniers travaux de fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Obtenir les détails d’un travail de fine-tuning spécifique
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annuler un travail de fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Lister jusqu’à n événements du travail
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

La première étape du processus est _la validation du fichier d’entraînement_ pour s’assurer que les données sont au bon format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Étape 2.3 : Suivre les événements pour surveiller les progrès


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Étape 2.4 : Voir le statut dans le tableau de bord OpenAI


Vous pouvez également consulter le statut en visitant le site Web d'OpenAI et en explorant la section _Fine-tuning_ de la plateforme. Cela vous montrera le statut du travail en cours, et vous permettra également de suivre l'historique des exécutions des travaux précédents. Dans cette capture d'écran, vous pouvez voir que l'exécution précédente a échoué, et que la deuxième exécution a réussi. Pour contexte, cela s'est produit lorsque la première exécution utilisait un fichier JSON avec des enregistrements mal formatés - une fois corrigé, la deuxième exécution s'est terminée avec succès et a rendu le modèle disponible à l'utilisation.

![Fine-tuning job status](../../../../../translated_images/fr/fine-tuned-model-status.563271727bf7bfba.webp)


Vous pouvez également consulter les messages d'état et les métriques en faisant défiler plus bas dans le tableau de bord visuel comme indiqué :

| Messages | Métriques |
|:---|:---|
| ![Messages](../../../../../translated_images/fr/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/fr/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Étape 3.1 : Récupérer l'ID et tester le modèle affiné dans le code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Étape 3.2 : Charger et tester le modèle affiné dans le Playground

Vous pouvez maintenant tester le modèle affiné de deux manières. Tout d'abord, vous pouvez visiter le Playground et utiliser le menu déroulant Models pour sélectionner votre modèle affiné parmi les options listées. L'autre option est d'utiliser l'option "Playground" affichée dans le panneau Fine-tuning (voir capture d'écran ci-dessus) qui lance la vue _comparative_ suivante montrant les versions du modèle de base et du modèle affiné côte à côte pour une évaluation rapide.

![Fine-tuning job status](../../../../../translated_images/fr/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Il suffit de remplir le contexte système utilisé dans vos données d'entraînement et de fournir votre question de test. Vous remarquerez que les deux côtés sont mis à jour avec le même contexte et la même question. Lancez la comparaison et vous verrez la différence dans les réponses entre les deux. _Notez comment le modèle affiné rend la réponse dans le format que vous avez fourni dans vos exemples tandis que le modèle de base suit simplement l'invite système_.

![Fine-tuning job status](../../../../../translated_images/fr/fine-tuned-playground-launch.5a26495c983c6350.webp)

Vous remarquerez que la comparaison fournit également le nombre de tokens pour chaque modèle, ainsi que le temps pris pour l'inférence. **Cet exemple spécifique est simpliste et destiné à montrer le processus sans refléter un jeu de données ou un scénario réel**. Vous pouvez constater que les deux échantillons montrent le même nombre de tokens (le contexte système et l'invite utilisateur sont identiques) avec le modèle affiné prenant plus de temps pour l'inférence (modèle personnalisé).

Dans des scénarios réels, vous n'utiliserez pas un exemple simplifié comme celui-ci, mais un affinage sur des données réelles (par exemple, un catalogue produit pour le service client) où la qualité de la réponse sera bien plus évidente. Dans _ce_ contexte, obtenir une qualité de réponse équivalente avec le modèle de base nécessitera plus d'ingénierie de prompt personnalisée, ce qui augmentera l'utilisation des tokens et potentiellement le temps de traitement lié à l'inférence. _Pour essayer cela, consultez les exemples de fine-tuning dans l'OpenAI Cookbook pour commencer._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant foi. Pour les informations critiques, une traduction professionnelle réalisée par un humain est recommandée. Nous déclinons toute responsabilité en cas de malentendus ou de mauvaises interprétations résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
